Additional verification of truth tables by results

Method 1

In [53]:
import itertools
import networkx as nx

def _OR(vals): 
    return int(any(vals))

def _NOR(vals):
    return int(not any(vals))

def _NOT(x):
    return 1 - int(x)

def truth_table_from_nx_schema_A(G, input_order=None, output_order=None):
    """
    Evaluate a combinational DAG G with the following rules:
      - node['type'] == 'input'  => primary input
      - node['type'] == 'output' => OR gate of predecessors
      - otherwise:
          * if fan-in == 1 -> NOT
          * if fan-in >= 2 -> NOR (NOT of OR)
    Returns: (header, rows) where rows are lists of bits [inputs..., outputs...]
    """
    if not nx.is_directed_acyclic_graph(G):
        raise ValueError("Graph must be a DAG")

    # Collect inputs/outputs
    inputs = [n for n, d in G.nodes(data=True) if d.get('type') == 'input']
    outputs = [n for n, d in G.nodes(data=True) if d.get('type') == 'output']

    if not inputs:
        raise ValueError("No primary inputs found (nodes with {'type': 'input'})")
    if not outputs:
        raise ValueError("No primary outputs found (nodes with {'type': 'output'})")

    # Stable ordering
    def _key(n): 
        try: return (0, int(n))
        except: return (1, str(n))
    inputs = sorted(inputs, key=_key) if input_order is None else list(input_order)
    outputs = sorted(outputs, key=_key) if output_order is None else list(output_order)

    topo = list(nx.topological_sort(G))
    rows = []

    for bits in itertools.product([0, 1], repeat=len(inputs)):
        val = {}

        # Seed PI values
        for n, b in zip(inputs, bits):
            val[n] = int(b)

        # Evaluate others in topo order
        for n in topo:
            if n in val:  # already set (input)
                continue
            preds = list(G.predecessors(n))
            t = G.nodes[n].get('type', None)

            if t == 'input':
                # already seeded; if graph provided a value elsewhere, keep it
                val.setdefault(n, 0)
            elif t == 'output':
                if len(preds) == 0:
                    raise ValueError(f"Output node {n} has no predecessors")
                vals = [val[p] for p in preds]
                val[n] = _OR(vals)
            else:
                # internal gate by fan-in
                if len(preds) == 0:
                    raise ValueError(f"Internal node {n} has no predecessors (constant not supported)")
                elif len(preds) == 1:
                    val[n] = _NOT(val[preds[0]])
                else:
                    vals = [val[p] for p in preds]
                    val[n] = _NOR(vals)

        # Collect outputs
        out_bits = [val[o] for o in outputs]
        rows.append(list(bits) + out_bits)

    header = [str(n) for n in inputs + outputs]
    return header, rows

def print_truth_table(header, rows):
    # pretty printer
    colw = [max(len(h), 1) for h in header]
    for r in rows:
        for i, x in enumerate(r):
            colw[i] = max(colw[i], len(str(x)))
    line = "| " + " | ".join(h.rjust(colw[i]) for i, h in enumerate(header)) + " |"
    sep  = "|-" + "-|-".join("-" * colw[i] for i in range(len(header))) + "-|"
    print(line)
    print(sep)
    for r in rows:
        print("| " + " | ".join(str(x).rjust(colw[i]) for i, x in enumerate(r)) + " |")


Characterize Figure 2 main circuit

In [77]:
from pathlib import Path
import pickle

circuit_hex = '0x52'

run_dir = f"/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/manuscript/Figures/Figure 2/Diversity panels/Notebook analysis/{circuit_hex}/seed_1/"


output_dir = Path(run_dir) / "optimal_topologies"
pkl_path = output_dir / "optimal_topologies.pkl"

if not pkl_path.exists():
    raise FileNotFoundError(f"Missing file: {pkl_path}")

with open(pkl_path, "rb") as f:
    unique_graphs = pickle.load(f)

print(f"Loaded {len(unique_graphs)} graphs")

Loaded 7 graphs


In [55]:
for node in unique_graphs[4].nodes(data=True):
    print(node)

(0, {'type': 'input'})
(1, {'type': 'input'})
(2, {'type': 'input'})
(11, {'type': 'output'})
(12, {})
(17, {})
(18, {})
(19, {})
(20, {})
(21, {})


In [79]:
for G in unique_graphs:
    header, rows = truth_table_from_nx_schema_A(G)
    print_truth_table(header, rows)
    print("\n")

| 0 | 1 | 2 | 8 |
|---|---|---|---|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 0 |


| 0 | 1 | 2 | 8 |
|---|---|---|---|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 0 |


| 0 | 1 | 2 | 8 |
|---|---|---|---|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 0 |




In [85]:
output = []
for row in rows:
    print(row[-1])
    output.append(row[-1])

0
0
0
0
1
0
1
0


In [81]:
output

[0, 0, 0, 0, 1, 0, 1, 0]

In [82]:
binary_string = "".join(map(str, output))
binary_string

'00001010'

In [83]:
decimal = int(binary_string, 2)
decimal

10

In [ ]:
nibbles = (len(binary_string) + 3) // 4
nibbles

2

In [114]:
hex_string = f"0x{decimal:0{nibbles}X}"
hex_string

'0x0A'

In [115]:
f"0x{decimal:02X}"

'0x0A'

In [116]:
hex_string == circuit_hex

True

#### Characterization of all circuits

Hex values

In [144]:
#29 and 49, 86, 92 missing? 

circuits_hex_list = [
"0x03",
"0x06",
"0x09",
"0x0A",
"0x13",
"0x18",
"0x21",
"0x23",
"0x24",
"0x2A",
"0x2C",
"0x2F",
"0x35",
"0x38",
"0x3A",
"0x3B",
"0x42",
"0x4C",
"0x52",
"0x55",
"0x56",
"0x58",
"0x60",
"0x61",
"0x63",
"0x68",
"0x6D",
"0x6F",
"0x76",
"0x83",
"0x85",
"0x8B",
"0x8C",
"0x90",
"0x91",
"0x99",
"0x9E",
"0xA2",
"0xA4",
"0xA5",
"0xA6",
"0xAB",
"0xB6",
"0xB7",
"0xBA",
"0xBC",
"0xC2",
"0xC4",
"0xC7",
"0xD0",
"0xD2",
"0xDA",
"0xDC",
"0xE0",
"0xE6",
"0xEF",
"0xF0",
"0xF1",
"0xFD",
]

for circuit_hex in circuits_hex_list:

    run_dir = f"/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/manuscript/scratch_training/3in_registry_processed/{circuit_hex}/seed_1"

    output_dir = Path(run_dir) / "optimal_topologies"
    pkl_path = output_dir / "optimal_topologies.pkl"

    if not pkl_path.exists():
        raise FileNotFoundError(f"Missing file: {pkl_path}")

    with open(pkl_path, "rb") as f:
        unique_graphs = pickle.load(f)

    print(f"Loaded {len(unique_graphs)} graphs")

    num_failed_tests = 0
    for G in unique_graphs:
        header, rows = truth_table_from_nx_schema_A(G)
        #print_truth_table(header, rows)
        #print("\n")
        output = []
        for row in rows:
            #print(row[-1])
            output.append(row[-1])   
        binary_string = "".join(map(str, output)) 
        decimal = int(binary_string, 2)
        nibbles = (len(binary_string) + 3) // 4
        
        hex_string = f"0x{decimal:0{nibbles}X}"
        #print(hex_string)
        if hex_string == circuit_hex:
            pass
            #print("Circuit passed test")
        else:
            print("Circuit failed test")
            num_failed_tests = num_failed_tests + 1
    if num_failed_tests == 0:
        print(f"All circuits passed for {circuit_hex}")
    else:    
        print(f"Circuit failed tests for circuit {circuit_hex}: {num_failed_tests}")
            

Loaded 1 graphs
All circuits passed for 0x03
Loaded 2 graphs
All circuits passed for 0x06
Loaded 4 graphs
All circuits passed for 0x09
Loaded 3 graphs
All circuits passed for 0x0A
Loaded 1 graphs
All circuits passed for 0x13
Loaded 1 graphs
All circuits passed for 0x18
Loaded 4 graphs
All circuits passed for 0x21
Loaded 2 graphs
All circuits passed for 0x23
Loaded 1 graphs
All circuits passed for 0x24
Loaded 1 graphs
All circuits passed for 0x2A
Loaded 4 graphs
All circuits passed for 0x2C
Loaded 1 graphs
All circuits passed for 0x2F
Loaded 2 graphs
All circuits passed for 0x35
Loaded 4 graphs
All circuits passed for 0x38
Loaded 1 graphs
All circuits passed for 0x3A
Loaded 1 graphs
All circuits passed for 0x3B
Loaded 1 graphs
All circuits passed for 0x42
Loaded 1 graphs
All circuits passed for 0x4C
Loaded 7 graphs
All circuits passed for 0x52
Loaded 1 graphs
All circuits passed for 0x55
Loaded 1 graphs
All circuits passed for 0x56
Loaded 2 graphs
All circuits passed for 0x58
Loaded 3 g

In [143]:
circuits_hex_list = [
"0x000D",
"0x0239",
"0x0304",
"0x040B",
"0x0575",
"0x057A",
"0x0643",
"0x0760",
"0x09AF",
"0x0F42",
"0x1038",
"0x1048",
"0x10C9",
"0x1284",
"0x1323",
"0x13CE",
"0x1714",
"0x1858",
"0x1A60",
"0x1AC6",
"0x1CBF",
"0x1D95",
"0x1FDE",
"0x226B",
"0x22C6",
"0x23A7",
"0x240F",
"0x2A38",
"0x2A56",
"0x2FC7",
"0x3060",
"0x30CE",
"0x32AA",
"0x35C3",
"0x36DC",
"0x3812",
"0x3A17",
"0x3B31",
"0x3B60",
"0x3B68",
"0x409B",
"0x41A2",
"0x41B2",
"0x429B",
"0x4724",
"0x47FD",
"0x48C1",
"0x4A32",
"0x4BF8",
"0x5215",
"0x53AF",
"0x53D7",
"0x599A",
"0x5AAD",
"0x5B30",
"0x5DA9",
"0x5F01",
"0x5FE2",
"0x616A",
"0x648B",
"0x6572",
"0x680A",
"0x6847",
"0x699D",
"0x6F2A",
"0x7096",
"0x70EC",
"0x7176",
"0x822B",
"0x850E",
"0x8F63",
"0x914C",
"0x918A",
"0x93AC",
"0x9591",
"0x96F7",
"0x9917",
"0x9BF5",
"0x9F8A",
"0xA2DA",
"0xA7B2",
"0xA960",
"0xB744",
"0xB8AD",
"0xBC16",
"0xBCA3",
"0xBDF1",
"0xBEE9",
"0xBF36",
"0xC248",
"0xC4B2",
"0xC766",
"0xCB82",
"0xCBD6",
"0xCE97",
"0xD319",
"0xD326",
"0xD477",
"0xD4E4",
"0xD550",
"0xDA80",
"0xDBFA",
"0xE605",
"0xE677",
"0xE93A",
"0xECF1",
"0xEFEB",
"0xF43F",
"0xF4E7",
"0xF5A4",
"0xFC79"]

for circuit_hex in circuits_hex_list:

    run_dir = f"/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/manuscript/scratch_training/4in_registry_processed/{circuit_hex}/seed_1"

    output_dir = Path(run_dir) / "optimal_topologies"
    pkl_path = output_dir / "optimal_topologies.pkl"

    if not pkl_path.exists():
        raise FileNotFoundError(f"Missing file: {pkl_path}")

    with open(pkl_path, "rb") as f:
        unique_graphs = pickle.load(f)

    print(f"Loaded {len(unique_graphs)} graphs")

    num_failed_tests = 0
    for G in unique_graphs:
        header, rows = truth_table_from_nx_schema_A(G)
        #print_truth_table(header, rows)
        #print("\n")
        output = []
        for row in rows:
            #print(row[-1])
            output.append(row[-1])   
        binary_string = "".join(map(str, output)) 
        decimal = int(binary_string, 2)
        nibbles = (len(binary_string) + 3) // 4
        
        hex_string = f"0x{decimal:0{nibbles}X}"
        #print(hex_string)
        if hex_string == circuit_hex:
            pass
            #print("Circuit passed test")
        else:
            print("Circuit failed test")
            num_failed_tests = num_failed_tests + 1
    if num_failed_tests == 0:
        print(f"All circuits passed for {circuit_hex}")
    else:    
        print(f"Circuit failed tests for circuit {circuit_hex}: {num_failed_tests}")
            

Loaded 3 graphs
All circuits passed for 0x000D
Loaded 11 graphs
All circuits passed for 0x0239
Loaded 2 graphs
All circuits passed for 0x0304
Loaded 3 graphs
All circuits passed for 0x040B
Loaded 1 graphs
All circuits passed for 0x0575
Loaded 8 graphs
All circuits passed for 0x057A
Loaded 1 graphs
All circuits passed for 0x0643
Loaded 1 graphs
All circuits passed for 0x0760
Loaded 1 graphs
All circuits passed for 0x09AF
Loaded 10 graphs
All circuits passed for 0x0F42
Loaded 1 graphs
All circuits passed for 0x1038
Loaded 3 graphs
All circuits passed for 0x1048
Loaded 6 graphs
All circuits passed for 0x10C9
Loaded 2 graphs
All circuits passed for 0x1284
Loaded 10 graphs
All circuits passed for 0x1323
Loaded 2 graphs
All circuits passed for 0x13CE
Loaded 3 graphs
All circuits passed for 0x1714
Loaded 1 graphs
All circuits passed for 0x1858
Loaded 1 graphs
All circuits passed for 0x1A60
Loaded 1 graphs
All circuits passed for 0x1AC6
Loaded 1 graphs
All circuits passed for 0x1CBF
Loaded 4 g

Method 2

In [159]:
# ============================================================
# Digital-circuit truth tables from a NetworkX DAG
# Schema rules:
#   - node['type'] == 'input'  -> primary input
#   - node['type'] == 'output' -> OR of its predecessors
#   - all other nodes:
#       * 1 predecessor -> NOT
#       * 2+ predecessors -> NOR = NOT(OR(...))
# Edges are from driver -> load (signal direction).
# ============================================================

import itertools
import networkx as nx

# -----------------------------
# Pretty table printer (shared)
# -----------------------------
def print_truth_table(header, rows):
    colw = [max(len(str(h)), 1) for h in header]
    for r in rows:
        for i, x in enumerate(r):
            colw[i] = max(colw[i], len(str(x)))
    line = "| " + " | ".join(str(h).rjust(colw[i]) for i, h in enumerate(header)) + " |"
    sep  = "|-" + "-|-".join("-" * colw[i] for i in range(len(header))) + "-|"
    print(line); print(sep)
    for r in rows:
        print("| " + " | ".join(str(x).rjust(colw[i]) for i, x in enumerate(r)) + " |")


# ============================================================
# METHOD A — Direct evaluation over the DAG
# ============================================================
def _OR(vals): 
    return int(any(vals))

def _NOR(vals):
    return int(not any(vals))

def _NOT(x):
    return 1 - int(x)

def truth_table_from_nx_schema_A(G, input_order=None, output_order=None):
    """
    Evaluate a combinational DAG G using the user's schema.
    Returns: (header, rows) where rows are [inputs..., outputs...].
    """
    if not nx.is_directed_acyclic_graph(G):
        raise ValueError("Graph must be a DAG")

    # Collect I/Os
    inputs  = [n for n, d in G.nodes(data=True) if d.get('type') == 'input']
    outputs = [n for n, d in G.nodes(data=True) if d.get('type') == 'output']
    if not inputs:
        raise ValueError("No primary inputs found (nodes with {'type': 'input'})")
    if not outputs:
        raise ValueError("No primary outputs found (nodes with {'type': 'output'})")

    def _key(n):
        try: return (0, int(n))
        except: return (1, str(n))

    inputs  = (list(input_order)  if input_order  is not None else sorted(inputs,  key=_key))
    outputs = (list(output_order) if output_order is not None else sorted(outputs, key=_key))

    topo = list(nx.topological_sort(G))
    rows = []

    for bits in itertools.product([0, 1], repeat=len(inputs)):
        val = {}

        # Seed PIs
        for n, b in zip(inputs, bits):
            val[n] = int(b)

        # Propagate
        for n in topo:
            if n in val:  # input already set
                continue
            preds = list(G.predecessors(n))
            t = G.nodes[n].get('type')

            if t == 'input':
                # already seeded
                val.setdefault(n, 0)
            elif t == 'output':
                if not preds:
                    raise ValueError(f"Output node {n} has no predecessors")
                vals = [val[p] for p in preds]
                val[n] = _OR(vals)
            else:
                # internal gate
                if len(preds) == 0:
                    raise ValueError(f"Internal node {n} has no predecessors (constants not supported)")
                elif len(preds) == 1:
                    val[n] = _NOT(val[preds[0]])
                else:
                    vals = [val[p] for p in preds]
                    val[n] = _NOR(vals)

        out_bits = [val[o] for o in outputs]
        rows.append(list(bits) + out_bits)

    header = [str(n) for n in inputs + outputs]
    return header, rows


# ============================================================
# METHOD 2A — PyEDA expressions (preferred if available)
# ============================================================
# Robust import for exprvar across PyEDA versions
try:
    from pyeda.inter import exprvar    # preferred
    _HAVE_PYEDA = True
except Exception:
    try:
        from pyeda.boolalg.expr import exprvar  # fallback
        _HAVE_PYEDA = True
    except Exception:
        _HAVE_PYEDA = False

def nx_to_pyeda_exprs(G, input_order=None, output_order=None):
    """
    Convert DAG to PyEDA expressions per the schema.
    Returns (inputs_dict, outputs_dict) with string node IDs.
    """
    if not _HAVE_PYEDA:
        raise ImportError("PyEDA is not available. Install with `pip install pyeda`.")

    if not nx.is_directed_acyclic_graph(G):
        raise ValueError("Graph must be a DAG")

    inputs  = [n for n, d in G.nodes(data=True) if d.get('type') == 'input']
    outputs = [n for n, d in G.nodes(data=True) if d.get('type') == 'output']

    def _key(n):
        try: return (0, int(n))
        except: return (1, str(n))

    inputs  = (list(input_order)  if input_order  is not None else sorted(inputs,  key=_key))
    outputs = (list(output_order) if output_order is not None else sorted(outputs, key=_key))

    # variables
    X = {str(n): exprvar(str(n)) for n in inputs}

    cache = {}
    topo = list(nx.topological_sort(G))

    def node_expr(n):
        if n in cache: 
            return cache[n]
        t = G.nodes[n].get('type')
        preds = list(G.predecessors(n))

        if t == 'input':
            e = X[str(n)]
        elif t == 'output':
            if not preds:
                raise ValueError(f"Output node {n} has no predecessors")
            # OR of predecessors
            args = [node_expr(p) for p in preds]
            e = args[0]
            for a in args[1:]:
                e = e | a
        else:
            if len(preds) == 0:
                raise ValueError(f"Internal node {n} has no predecessors")
            elif len(preds) == 1:
                # NOT
                e = ~node_expr(preds[0])
            else:
                # NOR = NOT(OR(...))
                args = [node_expr(p) for p in preds]
                o = args[0]
                for a in args[1:]:
                    o = o | a
                e = ~o

        cache[n] = e
        return e

    outs = {str(o): node_expr(o) for o in outputs}
    return X, outs

def pyeda_tt_table(inputs_dict, outputs_dict):
    """
    Build a combined truth table for all outputs using PyEDA expressions,
    by evaluating each output under all input assignments via `restrict`.
    Returns (header, rows) so you can reuse print_truth_table.
    """
    # stable input/output ordering
    def _key(k):
        try: return (0, int(k))
        except: return (1, k)

    in_names  = sorted(inputs_dict.keys(), key=_key)
    out_names = sorted(outputs_dict.keys(), key=_key)

    header = in_names + out_names
    rows = []

    for bits in itertools.product([0, 1], repeat=len(in_names)):
        assign = {inputs_dict[name]: b for name, b in zip(in_names, bits)}
        out_bits = []
        for oname in out_names:
            ev = outputs_dict[oname].restrict(assign)
            # With a full assignment, ev should be constant
            if hasattr(ev, "is_one") and ev.is_one():
                out_bits.append(1)
            elif hasattr(ev, "is_zero") and ev.is_zero():
                out_bits.append(0)
            else:
                # Fallback: try bool conversion if available, else raise
                try:
                    out_bits.append(1 if bool(ev) else 0)
                except Exception:
                    raise RuntimeError(f"Could not fully evaluate output {oname} under assignment {assign}")
        rows.append(list(bits) + out_bits)

    return header, rows

def pyeda_truth_table(inputs_dict, outputs_dict):
    """
    Print a combined truth table using the helper above.
    """
    header, rows = pyeda_tt_table(inputs_dict, outputs_dict)
    print_truth_table(header, rows)

def pyeda_minimized(outputs_dict):
    """
    Simplified expressions (Espresso + algebraic).
    """
    for oname, e in outputs_dict.items():
        print(f"{oname} simplified: {e.simplify()}")


# ============================================================
# METHOD 2B — SymPy expressions (fallback if PyEDA isn't available)
# ============================================================
try:
    import sympy as sp
    _HAVE_SYMPY = True
except Exception:
    _HAVE_SYMPY = False

def nx_to_sympy_exprs(G, input_order=None, output_order=None):
    """
    Convert DAG to SymPy Boolean expressions per the schema.
    Returns (inputs_syms, outputs_exprs).
    """
    if not _HAVE_SYMPY:
        raise ImportError("SymPy is not available. Install with `pip install sympy`.")

    if not nx.is_directed_acyclic_graph(G):
        raise ValueError("Graph must be a DAG")

    inputs  = [n for n, d in G.nodes(data=True) if d.get('type') == 'input']
    outputs = [n for n, d in G.nodes(data=True) if d.get('type') == 'output']

    def _key(n):
        try: return (0, int(n))
        except: return (1, str(n))

    inputs  = (list(input_order)  if input_order  is not None else sorted(inputs,  key=_key))
    outputs = (list(output_order) if output_order is not None else sorted(outputs, key=_key))

    X = {str(n): sp.symbols(str(n)) for n in inputs}
    cache = {}
    topo = list(nx.topological_sort(G))

    def node_expr(n):
        if n in cache:
            return cache[n]
        t = G.nodes[n].get('type')
        preds = list(G.predecessors(n))

        if t == 'input':
            e = X[str(n)]
        elif t == 'output':
            if not preds:
                raise ValueError(f"Output node {n} has no predecessors")
            e = sp.Or(*[node_expr(p) for p in preds])
        else:
            if len(preds) == 0:
                raise ValueError(f"Internal node {n} has no predecessors")
            elif len(preds) == 1:
                e = sp.Not(node_expr(preds[0]))  # NOT
            else:
                e = sp.Not(sp.Or(*[node_expr(p) for p in preds]))  # NOR
        cache[n] = e
        return e

    outs = {str(o): node_expr(o) for o in outputs}
    return X, outs

def sympy_truth_table(inputs_dict, outputs_dict):
    """
    Combined truth table for all outputs.
    Returns (header, rows) -> use print_truth_table to display.
    """
    def _key(k):
        try: return (0, int(k))
        except: return (1, k)

    in_names  = sorted(inputs_dict.keys(), key=_key)
    out_names = sorted(outputs_dict.keys(), key=_key)

    header = in_names + out_names
    rows = []
    for bits in itertools.product([0,1], repeat=len(in_names)):
        subs = {inputs_dict[n]: bool(b) for n, b in zip(in_names, bits)}
        outs = [int(bool(outputs_dict[o].subs(subs))) for o in out_names]
        rows.append(list(bits) + outs)
    return header, rows

def sympy_simplified(outputs_dict, form='dnf'):
    """
    Print simplified expressions (DNF by default; use form='cnf' if you prefer).
    """
    for name, e in outputs_dict.items():
        print(f"{name} simplified: {sp.simplify_logic(e, form=form)}")


    
def natural_node_sort_key(n):
    """Sort nodes as integers when possible; else as strings."""
    try:
        return (0, int(n))
    except Exception:
        return (1, str(n))

def resolve_io_orders(G, desired_outputs=None, desired_inputs=None):
    """
    Returns (input_order, output_order) for a given graph G.

    - If desired_inputs is None: use all nodes with type='input' (sorted).
      Else: coerce to list and validate nodes exist.

    - If desired_outputs is None: use all nodes with type='output' (sorted).
      Else: coerce to list and validate nodes exist.
    """
    # Inputs
    if desired_inputs is None:
        inputs = [n for n, d in G.nodes(data=True) if d.get('type') == 'input']
        input_order = sorted(inputs, key=natural_node_sort_key)
    else:
        input_order = list(desired_inputs)
        missing = [n for n in input_order if n not in G.nodes]
        if missing:
            raise ValueError(f"Requested input(s) not found: {missing}")

    # Outputs
    if desired_outputs is None:
        outputs = [n for n, d in G.nodes(data=True) if d.get('type') == 'output']
        if not outputs:
            raise ValueError("No nodes with type='output' found")
        output_order = sorted(outputs, key=natural_node_sort_key)
    else:
        # allow a single node id or a list
        if not isinstance(desired_outputs, (list, tuple)):
            desired_outputs = [desired_outputs]
        output_order = list(desired_outputs)
        missing = [n for n in output_order if n not in G.nodes]
        if missing:
            raise ValueError(f"Requested output(s) not found: {missing}")

    return input_order, output_order

In [162]:
circuit_hex = '0x52'

run_dir = f"/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/manuscript/Figures/Figure 2/Diversity panels/Notebook analysis/{circuit_hex}/seed_1/"


output_dir = Path(run_dir) / "optimal_topologies"
pkl_path = output_dir / "optimal_topologies.pkl"

if not pkl_path.exists():
    raise FileNotFoundError(f"Missing file: {pkl_path}")

with open(pkl_path, "rb") as f:
    unique_graphs = pickle.load(f)

print(f"Loaded {len(unique_graphs)} graphs")    


DESIRED_OUTPUTS = None          # e.g., 11 or [11, 12]
DESIRED_INPUTS  = None          # e.g., [0, 1, 2]

for i, G in enumerate(unique_graphs, start=1):
    print(f"\n===== Circuit {i} =====")

    # Resolve per-graph I/O orders
    input_order, output_order = resolve_io_orders(
        G, desired_outputs=DESIRED_OUTPUTS, desired_inputs=DESIRED_INPUTS
    )

    # === METHOD A (direct eval) ===
    print("\n=== METHOD A (direct eval) ===")
    header, rows = truth_table_from_nx_schema_A(G, input_order=input_order, output_order=output_order)
    print_truth_table(header, rows)

    # === METHOD 2A (PyEDA) ===
    if _HAVE_PYEDA:
        print("\n=== METHOD 2A (PyEDA) ===")
        X, OUTS = nx_to_pyeda_exprs(G, input_order=input_order, output_order=output_order)
        # uses the version-safe truth table printer that evaluates via restrict
        pyeda_truth_table(X, OUTS)
        pyeda_minimized(OUTS)
    else:
        print("\n(PyEDA not installed; skipping METHOD 2A)")

    # === METHOD 2B (SymPy) ===
    if _HAVE_SYMPY:
        print("\n=== METHOD 2B (SymPy) ===")
        Xs, OUTs = nx_to_sympy_exprs(G, input_order=input_order, output_order=output_order)
        h, r = sympy_truth_table(Xs, OUTs)
        print_truth_table(h, r)
        sympy_simplified(OUTs, form='dnf')
    else:
        print("\n(SymPy not installed; skipping METHOD 2B)")


Loaded 7 graphs

===== Circuit 1 =====

=== METHOD A (direct eval) ===
| 0 | 1 | 2 | 11 |
|---|---|---|----|
| 0 | 0 | 0 |  0 |
| 0 | 0 | 1 |  1 |
| 0 | 1 | 0 |  0 |
| 0 | 1 | 1 |  1 |
| 1 | 0 | 0 |  0 |
| 1 | 0 | 1 |  0 |
| 1 | 1 | 0 |  1 |
| 1 | 1 | 1 |  0 |

=== METHOD 2A (PyEDA) ===
| 0 | 1 | 2 | 11 |
|---|---|---|----|
| 0 | 0 | 0 |  0 |
| 0 | 0 | 1 |  1 |
| 0 | 1 | 0 |  0 |
| 0 | 1 | 1 |  1 |
| 1 | 0 | 0 |  0 |
| 1 | 0 | 1 |  0 |
| 1 | 1 | 0 |  1 |
| 1 | 1 | 1 |  0 |
11 simplified: Not(Or(Not(Or(Not(Or(2, Not(Or(0, 2)))), Not(Or(0, Not(Or(0, 2)))))), Not(Or(1, Not(Or(0, Not(Or(0, 2))))))))

=== METHOD 2B (SymPy) ===
| 0 | 1 | 2 | 11 |
|---|---|---|----|
| 0 | 0 | 0 |  0 |
| 0 | 0 | 1 |  1 |
| 0 | 1 | 0 |  0 |
| 0 | 1 | 1 |  1 |
| 1 | 0 | 0 |  0 |
| 1 | 0 | 1 |  0 |
| 1 | 1 | 0 |  1 |
| 1 | 1 | 1 |  0 |
11 simplified: (2 & ~0) | (0 & 1 & ~2)

===== Circuit 2 =====

=== METHOD A (direct eval) ===
| 0 | 1 | 2 | 11 |
|---|---|---|----|
| 0 | 0 | 0 |  0 |
| 0 | 0 | 1 |  1 |
| 0 | 1 | 

In [163]:
Xs

{'0': 0, '1': 1, '2': 2}

In [165]:
circuits_hex_list = [
"0x000D",
"0x0239",
"0x0304",
"0x040B",
"0x0575",
"0x057A",
"0x0643",
"0x0760",
"0x09AF",
"0x0F42",
"0x1038",
"0x1048",
"0x10C9",
"0x1284",
"0x1323",
"0x13CE",
"0x1714",
"0x1858",
"0x1A60",
"0x1AC6",
"0x1CBF",
"0x1D95",
"0x1FDE",
"0x226B",
"0x22C6",
"0x23A7",
"0x240F",
"0x2A38",
"0x2A56",
"0x2FC7",
"0x3060",
"0x30CE",
"0x32AA",
"0x35C3",
"0x36DC",
"0x3812",
"0x3A17",
"0x3B31",
"0x3B60",
"0x3B68",
"0x409B",
"0x41A2",
"0x41B2",
"0x429B",
"0x4724",
"0x47FD",
"0x48C1",
"0x4A32",
"0x4BF8",
"0x5215",
"0x53AF",
"0x53D7",
"0x599A",
"0x5AAD",
"0x5B30",
"0x5DA9",
"0x5F01",
"0x5FE2",
"0x616A",
"0x648B",
"0x6572",
"0x680A",
"0x6847",
"0x699D",
"0x6F2A",
"0x7096",
"0x70EC",
"0x7176",
"0x822B",
"0x850E",
"0x8F63",
"0x914C",
"0x918A",
"0x93AC",
"0x9591",
"0x96F7",
"0x9917",
"0x9BF5",
"0x9F8A",
"0xA2DA",
"0xA7B2",
"0xA960",
"0xB744",
"0xB8AD",
"0xBC16",
"0xBCA3",
"0xBDF1",
"0xBEE9",
"0xBF36",
"0xC248",
"0xC4B2",
"0xC766",
"0xCB82",
"0xCBD6",
"0xCE97",
"0xD319",
"0xD326",
"0xD477",
"0xD4E4",
"0xD550",
"0xDA80",
"0xDBFA",
"0xE605",
"0xE677",
"0xE93A",
"0xECF1",
"0xEFEB",
"0xF43F",
"0xF4E7",
"0xF5A4",
"0xFC79"]

for circuit_hex in circuits_hex_list:

    run_dir = f"/home/gridsan/spalacios/Designing complex biological circuits with deep neural networks/manuscript/scratch_training/4in_registry_processed/{circuit_hex}/seed_1"

    output_dir = Path(run_dir) / "optimal_topologies"
    pkl_path = output_dir / "optimal_topologies.pkl"

    if not pkl_path.exists():
        raise FileNotFoundError(f"Missing file: {pkl_path}")

    with open(pkl_path, "rb") as f:
        unique_graphs = pickle.load(f)

    print(f"Loaded {len(unique_graphs)} graphs")

    num_failed_tests = 0
    for G in unique_graphs:
        
        input_order, output_order = resolve_io_orders(
            G, desired_outputs=DESIRED_OUTPUTS, desired_inputs=DESIRED_INPUTS
        )    
        X, OUTS = nx_to_pyeda_exprs(G, input_order=input_order, output_order=output_order)
        header, rows = pyeda_tt_table(X, OUTS)
        #print_truth_table(header, rows)
        #print("\n")
        output = []
        for row in rows:
            #print(row[-1])
            output.append(row[-1])   
        binary_string = "".join(map(str, output)) 
        decimal = int(binary_string, 2)
        nibbles = (len(binary_string) + 3) // 4
        
        hex_string = f"0x{decimal:0{nibbles}X}"
        #print(hex_string)
        if hex_string == circuit_hex:
            pass
            #print("Circuit passed test")
        else:
            print("Circuit failed test")
            num_failed_tests = num_failed_tests + 1
    if num_failed_tests == 0:
        print(f"All circuits passed for {circuit_hex}")
    else:    
        print(f"Circuit failed tests for circuit {circuit_hex}: {num_failed_tests}")

Loaded 3 graphs
All circuits passed for 0x000D
Loaded 11 graphs
All circuits passed for 0x0239
Loaded 2 graphs
All circuits passed for 0x0304
Loaded 3 graphs
All circuits passed for 0x040B
Loaded 1 graphs
All circuits passed for 0x0575
Loaded 8 graphs
All circuits passed for 0x057A
Loaded 1 graphs
All circuits passed for 0x0643
Loaded 1 graphs
All circuits passed for 0x0760
Loaded 1 graphs
All circuits passed for 0x09AF
Loaded 10 graphs
All circuits passed for 0x0F42
Loaded 1 graphs
All circuits passed for 0x1038
Loaded 3 graphs
All circuits passed for 0x1048
Loaded 6 graphs
All circuits passed for 0x10C9
Loaded 2 graphs
All circuits passed for 0x1284
Loaded 10 graphs
All circuits passed for 0x1323
Loaded 2 graphs
All circuits passed for 0x13CE
Loaded 3 graphs
All circuits passed for 0x1714
Loaded 1 graphs
All circuits passed for 0x1858
Loaded 1 graphs
All circuits passed for 0x1A60
Loaded 1 graphs
All circuits passed for 0x1AC6
Loaded 1 graphs
All circuits passed for 0x1CBF
Loaded 4 g